In [79]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

from misc import *
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [80]:
# 定义稀疏自编码器
class SparseAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size, sparsity_param=0.05, beta=3):
        super(SparseAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size, input_size),
            nn.Sigmoid()
        )
        self.sparsity_param = sparsity_param  # 稀疏性目标
        self.beta = beta  # KL散度惩罚项的权重

    def forward(self, x):
        hidden = self.encoder(x)
        output = self.decoder(hidden)
        return output, hidden


In [81]:
# 定义卷积稀疏自编码器
class ConvSparseAutoencoder(nn.Module):
    def __init__(self, sparsity_param=0.05, beta=3):
        super(ConvSparseAutoencoder, self).__init__()
        
        # 编码器部分：卷积层提取特征
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),  # 1个输入通道（如灰度图），16个输出特征图
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),  # 16个输入特征图，32个输出特征图
            nn.ReLU()
        )
        
        # 解码器部分：反卷积（转置卷积）层重建输入
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),  # 从32个特征图还原到16个
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, padding=1, output_padding=1),  # 从16个特征图还原到1个通道
            nn.Sigmoid()
        )
        
        self.sparsity_param = sparsity_param  # 稀疏性目标
        self.beta = beta  # KL散度惩罚项的权重

    def forward(self, x):
        hidden = self.encoder(x)
        output = self.decoder(hidden)
        return output, hidden

    def kl_divergence(self, rho, rho_hat):
        """KL散度，用于稀疏性惩罚"""
        rho = torch.tensor(rho)
        rho_hat = torch.mean(rho_hat, dim=0)
        return torch.sum(rho * torch.log(rho / rho_hat) + (1 - rho) * torch.log((1 - rho) / (1 - rho_hat)))

In [82]:
dataset_path = 'G:/MRM_0.5/'
train_data_path = dataset_path + 'train/'
test_data_path = dataset_path + 'test/'

def preprocessing(data, snr, pad_size, sparse_size = 8):

    hrrp, noise_power = awgn(data['hrrp']['HH'], snr=snr)
    hrrp = fftshift(ifft(hrrp,axis = 0),axes=0)
    hrrp = np.log10(np.abs(hrrp))
    return normalize(hrrp).astype(np.float32)

class Dataset(Dataset):
    def __init__(self, dataset_dir, snr, pad_size):
        self.snr = snr
        self.pad_size = pad_size
        self.dataset_dir = dataset_dir
        self.instance_list = self.get_instance()
    
    def get_instance(self):
        instance_list = []
        for label in os.listdir(self.dataset_dir):
            label_dir = os.path.join(self.dataset_dir,label)
            label_list = glob.glob(label_dir+'/*.pkl')
            instance_list += label_list
        return instance_list

    def __len__(self):
        return len(self.instance_list)

    def __getitem__(self, idx):
        data = load_pkl(self.instance_list[idx])
        x = preprocessing(data, snr = self.snr, pad_size= self.pad_size)  
        y = data['target_id']
        return x, torch.tensor(y, dtype=torch.long)

In [72]:
snr = 0
pad_size = 201
train_dataset = Dataset(train_data_path, snr = snr, pad_size = pad_size)
test_dataset = Dataset(test_data_path, snr = snr, pad_size = pad_size)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

inp, label = train_dataset.__getitem__(0)

inp = torch.from_numpy(inp).to(device).reshape(-1)
# print(inp.shape)

In [77]:
# 定义稀疏自编码器模型
input_size = inp.shape[0]

hidden_size = 128  # 隐藏层大小
model = SparseAutoencoder(input_size, hidden_size).to(device)

outputs, hidden = model.forward(inp.unsqueeze(0))


def l1_sparsity_loss(hidden):
    # 计算隐藏层激活值的 L1 正则化
    return torch.sum(torch.abs(hidden))

# 定义损失函数，包括 MSE、L2 正则化 和 稀疏性正则化
def mse_sae_loss(outputs, inputs, model, hidden, lambda_l2, beta_sparsity, sparsity_param=0.05):
    # 计算 MSE 损失
    mse_loss = nn.MSELoss()(outputs, inputs)
    
    # 计算 L2 正则化（权重正则化）
    l2_reg = 0
    for param in model.parameters():
        l2_reg += torch.sum(param ** 2)
    
    # L1 稀疏性正则化
    # sparsity_loss = l1_sparsity_loss(hidden)
    
    # 总损失 = MSE + λ · L2 正则化 + β · L1 稀疏性正则化
    total_loss = mse_loss + lambda_l2 * l2_reg 
    # + beta_sparsity * sparsity_loss
    return total_loss 

In [78]:
# 定义优化器和损失函数
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
mse_loss = nn.MSELoss()
num_epochs = 1000
lambda_l2 = 0.5
beta_sparsity = 0.05
early_stop_threshold = 0
early_stop_patience = 1

# 稀疏自编码器训练过程
for epoch in range(num_epochs):
    total_loss = 0
    for inputs, _ in train_loader:
        inputs = inputs.view(inputs.size(0), -1).to(device)  # 展平输入图像
        optimizer.zero_grad()
        
        # 前向传播
        outputs, hidden = model(inputs)
        # 重构损失（MSE）
        loss = mse_sae_loss(outputs, inputs, model, hidden, lambda_l2, beta_sparsity, sparsity_param=0.05)
        # 反向传播与优化
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}')

    # 早停判断
    if total_loss < early_stop_threshold:
        early_stop_counter += 1
        if early_stop_counter >= early_stop_patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break
    else:
        early_stop_counter = 0  # 重置计数器
    
    scheduler.step()

Epoch [1/1000], Loss: 433141.2793
Epoch [2/1000], Loss: 338.8436
Early stopping triggered at epoch 2


In [75]:
torch.save(model, 'SAE_HRRP.pth')

In [65]:
for inputs, _ in train_loader:
    model.eval()
    inputs = inputs.view(inputs.size(0), -1).to(device)
    outputs, hiddens = model(inputs)
    break

In [76]:
hiddens[1]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [67]:
outputs

tensor([[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        ...,
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)